# Houston We Have a Programmer: A Crash Course in Lua

One of the defining aspects of Torch is that one interfaces with it uses the Lua language.

"What sort of spacey person would choose Lua over Python?!?" you ask.

Excellent question! The short answer is that Lua is lightweight and has a [stellar interpreter](http://luajit.org/), that rivals the performance of C and provides easy access to compiled C and CUDA methods. The result is that, instead of defining a computation in a slow language (like Python) and running it in another language (c.f. Theano, TensorFlow), the Lua code that you write is actually what is run! This has a bunch of nifty benefits, the most important of which is debuggability. More on this later, though... First we must learn some moonspeak!

## Lua Syntax & Semantics

In [1]:
print('Hello, world!') -- shout it from the moon!

Hello, world!	


Okay, so far so good! So far, we can tell that strings are surrounded by quotes (single or double), functions are called using the `()` operator, and comments are prefixed with `--`.

You might be thinking that Lua looks a bit like our good friend Python. Well, not quite... A more apt comparison would be to the syntax of Go with the semantics of JavaScript. On the plus side, though, a simple language is also simpler to learn. Let's forge on!

### Variables

Like C, Java, and JavaScript, Lua is [lexically scoped](https://en.wikipedia.org/wiki/Scope_%28computer_science%29#Lexical_scope_vs._dynamic_scope), which means that a local variable, defined using the `local` keyword is only bound within the smallest block that contains it. Similarly to JavaScript, a variable defined without using `local` (c.f. `var`) is global and can be accessed anywhere in the program; this is generally a Bad Thing&trade;.

In Lua, a block, very generally, is any control structure or definition that end with the keyword `end`. The following example demonstrates the `do ... end` block which does nothing other than define a block:

In [2]:
local a = 42
do
    print('a = '..a)
    b = 4
    local c = 5
end
print('b = '..b)

Okay, but what happens if we try to print out `c`?

In [3]:
print('c = '..c)

[string "print('c = '..c)..."]:1: attempt to concatenate global 'c' (a nil value)
stack traceback:
	[string "print('c = '..c)..."]:1: in main chunk
	[C]: in function 'xpcall'
	/Users/nhynes/torch/install/share/lua/5.1/itorch/main.lua:209: in function </Users/nhynes/torch/install/share/lua/5.1/itorch/main.lua:173>
	/Users/nhynes/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...rs/nhynes/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...rs/nhynes/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...rs/nhynes/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/nhynes/torch/install/share/lua/5.1/itorch/main.lua:381: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x010c48dd10: 

Cool, we get an error! Let's break it down a bit and, in doing so, notice something unique about Lua.

At first blush, it seems like Lua is complaining that `c` doesn't exist. That would be a very wrong interpretation, though. Looking more closely, it's saying that the problem is that we tried to concatenate a string with `nil`. What's more is that it states that `c` is some global variable! That can't be right... Ah ha! Here's the tricky part: **any undefined variable defaults to `nil`** and, since our local `c` has gone out of scope, the only option is a global variable that hasn't been defined!

The main takeaway, here, is that Lua will not complain about undefined variables and--even worse--treat them as `nil`, so if you're not careful, your program might be silently failing!

**Note:** a single line/program entered into the iTorch notebook or Torch REPL is implicitly enclosed in a block, so variables have to be defined globally to persist across lines.

Okay, now that we have somewhere to store our stuff, let's look at some of the stuff we can store!

### Functions

It'd be tough to write code that trains function approximators without having some way to define functions, themselves!

In [4]:
function trainBatch(net, data, labels, learningRate)
    local loss = forward(net, data, labels)
    backward(net, loss)
    updateParameters(net, learningRate)
    return loss
end

### Tables

We've saved the best for last: tables. Similarly to JavaScript in which everything is an Object, most things in Lua are tables. If you're coming from Python, you might think of them as a mash-up of a `dict` and a `list` with the extra ability to be used as objects.

In [5]:
t = {[0]=0, 1, 2, 3}    -- create a table with an array part holding [1,2,3] and a dict part holding 0=0
print('t[1] = '..t[1])  -- Lua uses 1-indexing!
print('The contiguous array part of t, starting at 1, has size '..#t)

t[1] = 1	
The contiguous array part of t, starting at 1, has size 3	


In [6]:
-- add some stuff to the dict part
t.rng = function() return 4 end
print(t.rng())

function t.someOtherFn()
    print('^ shorthand for defining functions in tables')
end

t['0'] = ''
print(t[0]) -- numbers are not auto-converted! (c.f. JavaScript)

4	
0	


#### OOP

If you were paying attention, you'll remember that I mentioned that tables are also used for OOP. The syntax here gets a bit weird but if you can get through this part, it's all downhill from here.

Really, the only thing that you need to know is that calling a table function (like `t.rng`, above), accessed with `:` instead of `.`, will pass *the table itself* as the first argument, `self`. This is similar to how Python works.

Fortunately, Torch makes it easy to define and use classes using [`torch.class`](https://github.com/torch/torch7/blob/master/doc/utility.md#torch.class):

In [3]:
require 'nn' -- import the neural network package

MyModule, parent = torch.class('nn.MyModule', 'nn.Module')

function MyModule:__init() -- same as MyModule.__init(self)
    parent.__init(self)
    self.learning = 'ALL the things!'
end

function MyModule:updateOutput(input)
    print('Learning '..self.learning)
end

mod = nn.MyModule() -- create a new nn.MyModule
mod:updateOutput(torch.rand(3, 3))

Learning ALL the things!	


Great, that's about as tough at it gets. Now on to the easy stuff.

### Control Flow: Conditionals

In [8]:
local aBool = 42
if aBool == '42' then
    print('Yay, strings!')
elseif aBool >= 42 then
    print('A big number!')
else
    print("I have no idea what's going on!")
end

### Control Flow: Loops

In [9]:
for i=3,5 do
    print(i)
end

A big number!	


In [10]:
t = {'a', 'b', [7]='you bet!'}
for k,v in pairs(t) do
    print('t['..k..'] = '..v)
end

3	
4	
5	


t[1] = a	
t[2] = b	
t[7] = you bet!	


As a final (and largely irrelevant) note, like `return` a `break` statement must be the final statement in a block (even if only a  `do...end` ) and the `continue` statement doesn't exist.

# 🎉 Congratulations, you made it! 🎉

You now know everything you need to start doing actual deep learning in Torch!